In [1]:
import os
os.chdir('..')
import pickle
import numpy as np
import pandas as pd
from preprocessing.functions import *

In [2]:
import pickle
with open('data/dictionaries/s2i.pckl','rb') as f:
    s2i = pickle.load(f)
with open('data/dictionaries/d2i.pckl','rb') as f:
    d2i = pickle.load(f)
with open('data/dictionaries/p2i.pckl','rb') as f:
    p2i = pickle.load(f)
with open('data/dictionaries/all2idx.pckl','rb') as f:
    a2i = pickle.load(f)

In [ ]:
# for demo
df_demo = pd.DataFrame(columns=['jid','agg','gender'])
years = [2014,2015]
months = np.arange(12)+1
for year in years[1:]:
    for month in months:
        file = '/home/data/EHR/raw/SAMPLING_NPS_200_%d.txt' %(year*100+month)
        df = pd.read_csv(file)
        df = df[['JID','AGG','SEX_TP_CD']].dropna() # bits we will take
        df.columns = ['jid','agg','gender']
        df_demo = df_demo.append(df)
df_demo = df_demo.drop_duplicates()

In [ ]:
with open('/home/data/EHR/db/%d_demo.pckl'%year,'wb') as f:
    pickle.dump(df_demo,f)
del df_demo

In [ ]:
# for main
df_main = pd.DataFrame(columns=['spec_id','jid','msick','msick_ab','msick_idx','date'])
years = [2014,2015]
months = np.arange(12)+1
for year in years[:1]:
    for month in months:
        file = '/home/data/EHR/raw/SAMPLING_NPS_200_%d.txt' %(year*100+month)
        df = pd.read_csv(file)
        # remove hospitalization cases
        df = df[(df['FOM_TP_CD']==31)|(df['FOM_TP_CD']==51)|(df['FOM_TP_CD']==81)|
           (df['FOM_TP_CD']==82)|(df['FOM_TP_CD']==83)|(df['FOM_TP_CD']==111)]
        df = df[['SPEC_ID_SNO','JID','MSICK_CD','RECU_TO_DD']].dropna() # bits we will take
        df.columns = ['spec_id','jid','msick','date']
        sick = [s[:3] for s in df['msick']]
        df['msick_ab'] = sick
        df['msick_idx'] = [a2i[s] for s in sick]
        df_main = df_main.append(df)

In [ ]:
with open('/home/data/EHR/db/%d_main.pckl'%year,'wb') as f:
    pickle.dump(df_main,f)

In [ ]:
# for sick
df_sick = pd.DataFrame(columns=['spec_id','sick','sick_ab','sick_idx'])
years = [2014,2015]
months = np.arange(12)+1
for year in years[:1]:
    for month in months:
        file = '/home/data/EHR/raw/SAMPLING_NPS_400_%d.txt' %(year*100+month)
        df = pd.read_csv(file)
        df = df[['SPEC_ID_SNO','SICK_CD']].dropna() # bits we will take        
        save_list = [i for i,x in enumerate(df['SICK_CD']) if len(x.strip())>1]
        df = df.iloc[save_list]
        df.columns = ['spec_id','sick']
        sick = [s[:3] for s in df['sick']]
        df['sick_ab'] = sick
        df['sick_idx'] = [a2i[s] for s in sick]
        df = df[df['sick_idx']!=0]
        df_sick = df_sick.append(df)

In [ ]:
# for treat
df_treat = pd.DataFrame(columns=['spec_id','treat'])
years = [2014,2015]
months = np.arange(12)+1
for year in years[:1]:
    for month in months:
        file = '/home/data/EHR/raw/SAMPLING_NPS_300_%d.txt' %(year*100+month)
        df = pd.read_csv(file)
        df = df[['SPEC_ID_SNO','DIV_CD']].dropna() # bits we will take        
        save_list = [i for i,v in enumerate(df['DIV_CD']) if len(v.strip())>1]
        df = df.iloc[save_list]
        df.columns = ['spec_id','treat']
        treat = [str(s)[:5] for s in df['treat']]
        tmp = []
        for s in treat:
            try:
                s = a2i[s]
            except KeyError:
                s = 0
            tmp.append(s)
        df['treat'] = tmp
        df = df[df['treat']!=0]
        df_treat = df_treat.append(df)

In [ ]:
# for pres
df_pres = pd.DataFrame(columns=['spec_id','pres'])
years = [2014,2015]
months = np.arange(12)+1
for year in years[:1]:
    for month in months:
        file = '/home/data/EHR/raw/SAMPLING_NPS_300_%d.txt' %(year*100+month)
        df = pd.read_csv(file)
        df = df[['SPEC_ID_SNO','GNL_NM_CD']].dropna() # bits we will take        
        save_list = [i for i,v in enumerate(df['GNL_NM_CD']) if len(v.strip())>1]
        df = df.iloc[save_list]
        df.columns = ['spec_id','pres']
        pres = [str(s)[:4] for s in df['pres']]
        tmp = []
        for s in pres:
            try:
                s = a2i[s]
            except KeyError:
                s = 0
            tmp.append(s)
        df['pres'] = tmp
        df = df[df['pres']!=0]
#         print(len(df))
        df_pres = df_pres.append(df)

In [ ]:
for year in years[:1]:
    for month in months:
        file = '/home/data/EHR/raw/SAMPLING_NPS_530_%d.txt' %(year*100+month)
        df = pd.read_csv(file)
#         print(len(df))
        df = df[['SPEC_ID_SNO','GNL_NM_CD']].dropna() # bits we will take        
        save_list = [i for i,v in enumerate(df['GNL_NM_CD']) if len(v.strip())>1]
        df = df.iloc[save_list]
#         print(len(df))
        df.columns = ['spec_id','pres']
        pres = [str(s)[:4] for s in df['pres']]
        tmp = []
        for s in pres:
            try:
                s = a2i[s]
            except KeyError:
                s = 0
            tmp.append(s)
        df['pres'] = tmp
        df = df[df['pres']!=0]
#         print(len(df))
        df_pres = df_pres.append(df)

In [ ]:
import pickle
with open('/home/data/EHR/db/%d_main.pckl'%year,'wb') as f:
    pickle.dump(df_main,f)
del df_main
with open('/home/data/EHR/db/%d_sick.pckl'%year,'wb') as f:
    pickle.dump(df_sick,f)
del df_sick
with open('/home/data/EHR/db/%d_treat.pckl'%year,'wb') as f:
    pickle.dump(df_treat,f)
del df_treat
with open('/home/data/EHR/db/%d_pres.pckl'%year,'wb') as f:
    pickle.dump(df_pres,f)
del df_pres

In [ ]:
import pickle
with open('/home/data/EHR/db/%d_main.pckl'%year,'rb') as f:
    df_main = pickle.load(f)
with open('/home/data/EHR/db/%d_pres.pckl'%year,'rb') as f:
    df_pres = pickle.load(f)
with open('/home/data/EHR/db/%d_sick.pckl'%year,'rb') as f:
    df_sick = pickle.load(f)
with open('/home/data/EHR/db/%d_treat.pckl'%year,'rb') as f:
    df_treat = pickle.load(f)

In [ ]:
df_main = df_main[df_main.date<20160000]

In [ ]:
len(df_main)

In [ ]:
df_main = df_main.sort_values(['jid','date'],ascending=[True,True])
df_sick = df_sick.sort_values(['spec_id'],ascending=[True])
df_treat = df_treat.sort_values(['spec_id'],ascending=[True])
df_pres = df_pres.sort_values(['spec_id'],ascending=[True])

In [ ]:
import sqlite3
from pandas.io import sql

In [ ]:
# with open('/home/data/EHR/db/2014_main.pckl','rb') as f:
#     df_main = pickle.load(f)
with open('/home/data/EHR/db/2014_sick.pckl','rb') as f:
    df_sick = pickle.load(f)
with open('/home/data/EHR/db/2014_treat.pckl','rb') as f:
    df_treat = pickle.load(f)
with open('/home/data/EHR/db/2014_pres.pckl','rb') as f:
    df_pres = pickle.load(f)

In [ ]:
import time

In [ ]:
# # create in-memory database
# import time
# start = time.time()
# df_main.to_sql('main_table',con=con)
# print(time.time()-start)
# start = time.time()
# df_sick.to_sql('sick_table',con=con)
# print(time.time()-start)
# start = time.time()
# df_treat.to_sql('treat_table',con=con)
# print(time.time()-start)
# start = time.time()
# df_pres.to_sql('pres_table',con=con)
# print(time.time()-start)
# # create indices
# start = time.time()
# cur.execute('CREATE INDEX idx_main ON main_table(spec_id)')
# cur.execute('CREATE INDEX idx_sick ON sick_table(spec_id)')
# cur.execute('CREATE INDEX idx_treat ON treat_table(spec_id)')
# cur.execute('CREATE INDEX idx_pres ON pres_table(spec_id)')
# print(time.time()-start)
# con.commit()

In [ ]:
df_demo

In [ ]:
# create database
import time
years = [2014,2015]
for year in years:
    con = sqlite3.connect('/home/data/EHR/db/%d.db'%year)
    cur = con.cursor()
    con.text_factory = str
    with open('/home/data/EHR/db/%d_demo.pckl'%year,'rb') as f:
        df_main = pickle.load(f)
    start = time.time()
    cur.execute('DROP TABLE demo_table')
    df_main.to_sql('demo_table',con=con)
    print(time.time()-start)
    # create indices
    start = time.time()
    cur.execute('CREATE INDEX idx_demo ON demo_table(jid)')
    cur.execute('CREATE INDEX idx_demo2 ON demo_table(agg,gender)')
    print(time.time()-start)
    con.commit()
    del df_main
    con.close()

In [ ]:
# create database
import time
years = [2014,2015]
for year in years:
    con = sqlite3.connect('/home/data/EHR/db/%d.db'%year)
    cur = con.cursor()
    con.text_factory = str
    with open('/home/data/EHR/db/%d_main.pckl'%year,'rb') as f:
        df_main = pickle.load(f)
    with open('/home/data/EHR/db/%d_pres.pckl'%year,'rb') as f:
        df_pres = pickle.load(f)
    with open('/home/data/EHR/db/%d_sick.pckl'%year,'rb') as f:
        df_sick = pickle.load(f)
    with open('/home/data/EHR/db/%d_treat.pckl'%year,'rb') as f:
        df_treat = pickle.load(f)
    start = time.time()
    df_main.to_sql('main_table',con=con)
    print(time.time()-start)
    start = time.time()
    df_sick.to_sql('sick_table',con=con)
    print(time.time()-start)
    start = time.time()
    df_treat.to_sql('treat_table',con=con)
    print(time.time()-start)
    start = time.time()
    df_pres.to_sql('pres_table',con=con)
    print(time.time()-start)
    # create indices
    start = time.time()
    cur.execute('CREATE INDEX idx_main ON main_table(spec_id)')
    cur.execute('CREATE INDEX idx_sick ON sick_table(spec_id)')
    cur.execute('CREATE INDEX idx_treat ON treat_table(spec_id)')
    cur.execute('CREATE INDEX idx_pres ON pres_table(spec_id)')
    print(time.time()-start)
    con.commit()
    del df_main
    del df_pres
    del df_sick
    del df_treat
    con.close()

In [ ]:
import sqlite3

In [ ]:
years = [2015]
for year in years:
    con = sqlite3.connect('/home/data/EHR/db/%d.db'%year)
    cur = con.cursor()
    con.text_factory = str
    # 1 - select all ids from main_table
    print('Select all ids from main table')
    cur.execute('SELECT jid,spec_id,date,msick_idx FROM main_table')
    rows = cur.fetchall()
    
    out_dict = dict()
    cnt = 0
    for row in rows:
        cnt+=1
        jid,id,date,msick_idx = row
        if jid not in out_dict:
            out_dict[jid] = []
        cur.execute('SELECT treat FROM treat_table WHERE spec_id is %d' %id)
        treat_list = [x[0] for x in cur.fetchall()]
        cur.execute('SELECT sick_idx FROM sick_table WHERE spec_id is %d' %id)
        sick_list = [x[0] for x in cur.fetchall()]
        cur.execute('SELECT pres FROM pres_table WHERE spec_id is %d' %id)
        pres_list = [x[0] for x in cur.fetchall()]
        input_list = list(set(treat_list+sick_list+pres_list))
        out_dict[jid].append((id,date,input_list,msick_idx))
        if cnt%100000==0:
            print(cnt)
            
    # sort list by dates (only realized now that they are unsorted)
    out_dict[jid] = sorted(out_dict[jid], key=lambda x: x[1])
    
    print("Save as dictionary")
    with open('/home/data/EHR/db/out_dict_%d.pckl'%year,'wb') as f:
        pickle.dump(out_dict,f)
    del out_dict
    con.close()

In [ ]:
with open('/home/data/EHR/db/out_dict_%d.pckl'%year,'wb') as f:
    pickle.dump(out_dict,f)

In [ ]:
import pickle

In [ ]:
len(out_dict)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# for demo
df_demo = pd.DataFrame(columns=['spec_id','jid','msick_ab','agg','gender'])
years = [2014,2015]
months = np.arange(12)+1
for year in years[:1]:
    for month in months:
        file = '/home/data/EHR/raw/SAMPLING_NPS_200_%d.txt' %(year*100+month)
        df = pd.read_csv(file)
#         print(df.columns)
        df = df[['SPEC_ID_SNO','JID','MSICK_CD','AGG','SEX_TP_CD']].dropna() # bits we will take
        df.columns = ['spec_id','jid','msick_ab','agg','gender']
        sick = [s[:3] for s in df['msick_ab']]
        df['msick_ab'] = sick
        df_demo = df_demo.append(df)

In [ ]:
import pickle
with open('/home/data/EHR/db/%d_demo.pckl' %year,'wb') as f:
    pickle.dump(df_demo,f)

In [ ]:
month